##### Demo

## Use input size 416:
- file cfg
- yolo3_weight.h5 when convert
- file train.py

In [1]:
import colorsys
import cv2
import dlib
# import face_recognition
from keras import backend as K
from keras.utils import multi_gpu_model
from keras.layers import Input
from keras.models import load_model

import numpy as np
import os
from PIL import Image, ImageFont, ImageDraw
from timeit import default_timer as timer

from yolo3.model import yolo_eval, yolo_body, tiny_yolo_body
from yolo3.utils import letterbox_image

# Super resolution
from model import Generator
from PIL import Image
import os
import time
from torch.autograd import Variable
from torchvision.transforms import ToTensor, ToPILImage
import torch

import tensorflow as tf
import matplotlib.pyplot as plt
import math
import inception_resnet_v1

Using TensorFlow backend.
/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.23) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
from imutils.video import WebcamVideoStream

In [3]:
# from threading import Thread
# import cv2
# import imutils
# class WebcamVideoStream:
#     def __init__(self, src=0):
#         self.stream = cv2.VideoCapture(src)
#         self.stream.set(3, 800)
#         self.stream.set(4, 600)
#         (self.grabbed, self.frame) = self.stream.read()
        
#         self.stopped = False
        
#     def start(self):
#         # Start the thread to read frames from the video stream
#         Thread(target=self.update, args=()).start()
#         return self
    
#     def update(self):
#         while True:
#             if self.stopped:
#                 return
            
#             (self.grabbed, self.frame) = self.stream.read()
            
#     def read(self):
#         return self.frame
    
#     def stop(self):
#         self.stopped = True  

In [4]:
tf.reset_default_graph() 
# sess = tf.InteractiveSession()
sess = tf.Session()
images_pl = tf.placeholder(tf.float32, shape=[None, 160, 160, 3], name='input_image')
images_norm = tf.map_fn(lambda frame: tf.image.per_image_standardization(frame), images_pl)
train_mode = tf.placeholder(tf.bool)
age_logits, gender_logits, _ = inception_resnet_v1.inference(images_norm, keep_probability=0.8,
                                                             phase_train=train_mode,
                                                             weight_decay=1e-5)

gender = tf.argmax(tf.nn.softmax(gender_logits), 1)
age_ = tf.cast(tf.constant([i for i in range(0, 101)]), tf.float32)
age = tf.reduce_sum(tf.multiply(tf.nn.softmax(age_logits), age_), axis=1)
init_op = tf.group(tf.global_variables_initializer(),
                   tf.local_variables_initializer())
sess.run(init_op)
saver = tf.train.Saver()
ckpt = tf.train.get_checkpoint_state("/home/neosai/Documents/projects/deep_face_recognition/weights/models_gender_and_age/")
if ckpt and ckpt.model_checkpoint_path:
    saver.restore(sess, ckpt.model_checkpoint_path)
    print("restore model!")
else:
    pass
    print("error")

faces = np.empty((1,160, 160, 3))

INFO:tensorflow:Restoring parameters from /home/neosai/Documents/projects/deep_face_recognition/weights/models_gender_and_age/model.ckpt-14001
restore model!


In [5]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
gpu_num = 1

In [6]:
# model_path = '../logs/human_pose_dataset_2388_size_416_batch_size_4/human_pose_dataset_2388_size_416_batch_size_4.h5'
# model_path = '../model_data/yolo_weights_default_416.h5'
anchors_path = '/home/neosai/Documents/projects/deep_face_recognition/weights/yolo_anchors.txt'
# classes_path = '../model_data/coco_classes.txt'
# classes_path = '../model_data/human_pose.txt'

model_path = '/home/neosai/Documents/projects/deep_face_recognition/weights/ep069-loss46.542-val_loss45.218.h5'
classes_path = '/home/neosai/Documents/projects/deep_face_recognition/weights/face.txt'

score = 0.3
iou = 0.2
model_image_size = (416, 416)
sess = K.get_session()

In [7]:
# Get class
classes_path = os.path.expanduser(classes_path)
with open(classes_path) as f:
    class_names = f.readlines()

class_names = [c.strip() for c in class_names]

# Anchors
anchors_path = os.path.expanduser(anchors_path)
with open(anchors_path) as f:
    anchors = f.readline()
anchors = [float(x) for x in anchors.split(',')]
anchors = np.array(anchors).reshape(-1, 2)

In [8]:
# Load model
model_path = os.path.expanduser(model_path)
assert model_path.endswith('.h5'), 'Keras model end with file .h5'

num_anchors = len(anchors)
num_classes = len(class_names)

is_tiny_version = num_anchors==6
try:
    yolo_model = load_model(model_path, compile=False)
except:
    if is_tiny_version:
        yolo_model = tiny_yolo_body(Input(shape=(None, None, 3)), num_anchors//2, num_classes)
    else:
        yolo_model = yolo_body(Input(shape=(None, None, 3)), num_anchors//3, num_classes)
    
    yolo_model.load_weights(model_path)
else:
    yolo_model.layers[-1].output_shape[-1] == num_anchors/len(yolo_model.output) * (num_classes + 5), 'Mismatch between model and given anchor and class sizes'
    
print("{} model, anchors, and classes loaded.".format(model_path))

/home/neosai/Documents/projects/deep_face_recognition/weights/ep069-loss46.542-val_loss45.218.h5 model, anchors, and classes loaded.


In [9]:
# ix, iy, ex, ey = -1, -1, -1, -1
# ix1, iy1, ix2, iy2 = -1, -1, -1, -1
# cap_from_stream = False

In [10]:
# def limit_mem():
#     K.get_session().close()
#     cfg = K.tf.ConfigProto()
#     cfg.gpu_options.allow_growth = True
#     K.set_session(K.tf.Session(config=cfg))

In [11]:
# def draw_rec(event, x, y, flags, param):
#     global ix, iy, ex, ey, drawing, mode

#     if event == cv2.EVENT_LBUTTONDOWN:
#         ix, iy = x, y

#     elif event == cv2.EVENT_LBUTTONUP:
#         ex, ey = x, y
#         cv2.rectangle(param, (ix, iy), (x, y), (0, 255, 0), 0)

In [12]:
# def draw_line(event, x, y, flags, param):
#     global ix1, iy1, ix2, iy2, drawing, mode
#     if event == cv2.EVENT_LBUTTONDOWN:
#         ix1, iy1 = x, y
#         print(ix1, iy1)

#     elif event == cv2.EVENT_LBUTTONUP:
#         ix2, iy2 = x, y
#         print(ix2, iy2)
#         cv2.line(param, (ix1, iy1), (x, y), (0, 0, 255), 2)

In [13]:
# def super_resolution_image(image, model):
#     image = Variable(ToTensor()(image), volatile=True).unsqueeze(0)
#     # image = image.cuda()
#     out = model(image)
#     out_img = ToPILImage()(out[0].data.cpu())
#     return out_img

In [14]:
# def get_crop_size(video_path):
#     cap = cv2.VideoCapture(video_path)
#     while cap.isOpened():
#         ret, frame = cap.read()
#         if cap_from_stream:
#             frame = cv2.resize(frame, (1280, 720))
#         cv2.namedWindow('draw_rectangle')
#         cv2.setMouseCallback('draw_rectangle', draw_rec, frame)
#         # cv2.setMouseCallback("draw_rectangle", draw_line, frame)
#         print("Choose your area of interest!")
#         while 1:
#             cv2.imshow('draw_rectangle', frame)
#             k = cv2.waitKey(1) & 0xFF
#             if k == ord('a'):
#                 cv2.destroyAllWindows()
#                 break
#         break

# def get_crop_size1(video_path):
#     cap = cv2.VideoCapture(video_path)
#     while cap.isOpened():
#         ret, frame = cap.read()
#         if cap_from_stream:
#             frame = cv2.resize(frame, (1280, 720))
#         cv2.namedWindow('draw_line')
#         cv2.setMouseCallback('draw_line', draw_line, frame)
#         print("Choose your area of interest!")
#         while 1:
#             cv2.imshow('draw_line', frame)
#             k = cv2.waitKey(1) & 0xFF
#             if k == ord('a'):
#                 cv2.destroyAllWindows()
#                 break
#         break

In [15]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')
video_capture = cv2.VideoCapture('/home/neosai/Documents/dataset/camera_supervisor/1_03_H_082018120000.avi')
# out = cv2.VideoWriter('/home/neosai/Documents/dataset/camera_supervisor/output.avi',fourcc, 20.0, (1280,960))

# out = cv2.VideoWriter('/home/neosai/Documents/dataset/camera_supervisor/1_02_H_082018120000.mp4',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (1280,960))

In [16]:
def super_resolution_image(image, model):
    image = Variable(ToTensor()(image), volatile=True).unsqueeze(0)
    # image = image.cuda()
    out = model(image)
    out_img = ToPILImage()(out[0].data.cpu())
    return out_img

In [17]:
input_image_shape = K.placeholder(shape=(2, ))
boxes, scores, classes = yolo_eval(yolo_model.output, anchors, len(class_names), input_image_shape, score_threshold=score, iou_threshold=iou)
num_frame = 0
font = cv2.FONT_HERSHEY_DUPLEX
id_number = 0
upscale_factor = 4
model_name = "/home/neosai/Documents/projects/deep_face_recognition/weights/netG_epoch_4_100.pth"
# limit_mem()
model = Generator(upscale_factor).eval()
# model.cuda()
model.load_state_dict(torch.load(model_name, map_location=lambda storage, loc: storage))
# Video capture
# video_capture = WebcamVideoStream(src=0).start()
while (video_capture.isOpened()):

# while True:
    num_frame += 1

    # Read video frame and flip camera
    ret, frame = video_capture.read()
    print(type(frame))
#     frame = cv2.flip(frame, 1)
    frame_process = np.copy(frame)
#     frame_process = frame_process[]
    
    frame_process = frame_process[200: 960, 375: 1280]

    # Detect state standing and sleeping and sitting
    image = Image.fromarray(frame_process)

    # Process detect hand and recognition furniture
    boxed_image = letterbox_image(image, tuple(reversed(model_image_size)))
    image_data = np.array(boxed_image, dtype='float32')
    
    image_data /= 255.
    image_data = np.expand_dims(image_data, 0)
    
    out_boxes, out_scores, out_classes = sess.run([boxes, scores, classes],
                                                 feed_dict={
                                                     yolo_model.input: image_data,
                                                     input_image_shape: [image.size[1], image.size[0]],
                                                     K.learning_phase(): 0
                                                 })

    for i, c in reversed(list(enumerate(out_classes))):
        predicted_class = class_names[c]
        box = out_boxes[i]
        score = out_scores[i]
        
        label = '{} {:.2f}'.format(predicted_class, score)
        
        top, left, bottom, right = box  
        top = max(0, np.floor(top + 0.5).astype('int32'))
        left = max(0, np.floor(left + 0.5).astype('int32'))
        bottom = min(image.size[1], np.floor(bottom + 0.5).astype('int32'))
        right = min(image.size[0], np.floor(right + 0.5).astype('int32'))
        
        left = int(left - (right - left) / 4)
        top = int(top - (bottom - top) / 4)
        right = int(right + (right - left) / 4)
        bottom = int(bottom + (bottom - top) / 4)
        
        left += 375
        right += 375
        top += 200
        bottom += 200
        
        if score > 0.50:
            image_crop = frame[top:bottom, left:right]
            image_crop = Image.fromarray(image_crop, 'RGB')
            image_crop = super_resolution_image(image_crop, model)
            image_crop_array = np.asarray(image_crop)
            face_male_resize = image_crop.resize((160, 160), Image.ANTIALIAS)
            face = np.array(face_male_resize)
            faces[0, :, :, :] = face
            age_predict, gender_predict = sess.run([age, gender], feed_dict={images_pl: faces, train_mode: False})
            label_age_gender = "{}, {}".format(int(age_predict), "Female" if gender_predict == 0 else "Male")
            id_number += 1
            name = "../image/id_{}, {}".format(id_number, label_age_gender)
            cv2.imwrite(name + ".jpg", image_crop_array)
            cv2.putText(frame, label_age_gender, (left + 6, bottom + 6), font, 1.0, (255, 0, 255), 1)
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 3)
        cv2.putText(frame, label, (left + 6, top - 6), font, 1.0, (255, 0, 255), 1)
        
        print(left, right, top, bottom)
        print(label)
        
    cv2.rectangle(frame, (375, 200), (1280, 960), (0, 0, 255), 3)
    cv2.imshow("Frame", frame)
#     out.write(frame)
    #
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

<class 'numpy.ndarray'>


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


714 822 488 584
face 0.66
<class 'numpy.ndarray'>
714 822 488 584
face 0.66
<class 'numpy.ndarray'>
1034 1095 256 328
face 0.37
716 821 487 583
face 0.65
<class 'numpy.ndarray'>
717 820 489 582
face 0.71
<class 'numpy.ndarray'>
723 821 486 583
face 0.76
<class 'numpy.ndarray'>
722 824 486 583
face 0.67
<class 'numpy.ndarray'>
1036 1096 257 330
face 0.46
732 828 481 587
face 0.60
<class 'numpy.ndarray'>
1037 1095 256 333
face 0.41
734 830 481 583
face 0.81
<class 'numpy.ndarray'>
1036 1096 256 331
face 0.42
735 831 480 585
face 0.77
<class 'numpy.ndarray'>
920 976 379 445
face 0.32
736 832 479 585
face 0.76
<class 'numpy.ndarray'>
1037 1097 255 335
face 0.31
738 830 479 584
face 0.80
<class 'numpy.ndarray'>
920 976 379 446
face 0.38
1038 1096 256 334
face 0.53


KeyboardInterrupt: 

## Test

In [ ]:
import cv2
import matplotlib.pyplot as plt

In [ ]:
img = cv2.imread("data/image/hanoi.jpg")

In [ ]:
plt.imshow(img)

In [ ]:
img_crop = img[0:800, 1000:1400]

In [ ]:
plt.imshow(img_crop)

In [ ]:
a = []

In [ ]:
if not a:
    print("true")
else:
    print("False")